In [1]:
import gurobipy as gp
from gurobipy import GRB

def solve_rsu_allocation_with_local(jobs, rsus, servers, J_RT, params):
    """
    Solves the joint RSU, Server, and Local execution allocation problem
    based on the model in improved_gurobi.pdf.
    """

    # --- 1. Unpack Parameters ---
    R_i = params['R_i']     # Reachable RSUs for job i
    B = params['B']         # Data size of job i
    E = params['E']         # MEC execution time of job i
    D = params['D']         # Deadline for job i
    I = params['I']         # Importance weight of job i
    Cu = params['Cu']       # Data upload rate of an RSU
    c_s = params['c_s']     # Unit computation cost on MEC
    gamma = params['gamma'] # Local execution slowdown factor
    alpha = params['alpha'] # Objective weight (latency)
    beta = params['beta']   # Objective weight (cost)
    M = params['M']         # Big-M constant
    epsilon = params['epsilon'] # Small epsilon constant

    # --- 2. Create Model ---
    m = gp.Model("RSU_Server_Local_Allocation")
    m.Params.OutputFlag = 0  # Turn off solver log
    
    # --- 3. Decision Variables ---
    
    # Y[i, r]: 1 if job i uploads via RSU r
    Y = m.addVars([(i, r) for i in jobs for r in R_i[i]], 
                  vtype=GRB.BINARY, name="Y")

    # Z[i, x]: 1 if job i executes on MEC server x
    Z = m.addVars(jobs, servers, vtype=GRB.BINARY, name="Z")

    # Y_local[i]: 1 if job i executes on its local server
    Y_local = m.addVars(jobs, vtype=GRB.BINARY, name="Y_local")

    # Time variables
    t_s_up = m.addVars(jobs, vtype=GRB.CONTINUOUS, lb=0, name="t_s_up")
    t_e_up = m.addVars(jobs, vtype=GRB.CONTINUOUS, lb=0, name="t_e_up")
    t_s_ex = m.addVars(jobs, vtype=GRB.CONTINUOUS, lb=0, name="t_s_ex")
    t_e_ex = m.addVars(jobs, vtype=GRB.CONTINUOUS, lb=0, name="t_e_ex")

    # Ordering Variables
    # delta_up[i, j, r]: 1 if job i uploads before job j on RSU r (for i != j)
    delta_up = m.addVars([(i, j, r) for i in jobs for j in jobs if i != j for r in rsus 
                          if (i, r) in Y.keys() and (j, r) in Y.keys()],
                         vtype=GRB.BINARY, name="delta_up")
    
    # delta_ex[i, j, x]: 1 if job i executes before job j on server x (for i != j)
    delta_ex = m.addVars([(i, j, x) for i in jobs for j in jobs if i != j for x in servers],
                         vtype=GRB.BINARY, name="delta_ex")

    # z_up[i, j, r]: 1 if both job i and j upload on RSU r (for i < j)
    z_up = m.addVars([(i, j, r) for i in jobs for j in jobs if i < j for r in rsus
                      if (i, r) in Y.keys() and (j, r) in Y.keys()],
                     vtype=GRB.BINARY, name="z_up")

    # z_ex[i, j, x]: 1 if both job i and j execute on server x (for i < j)
    z_ex = m.addVars([(i, j, x) for i in jobs for j in jobs if i < j for x in servers],
                     vtype=GRB.BINARY, name="z_ex")

    # --- 4. Constraints ---

    # -- Time Relation Constraints --
    
    # Upload duration
    for i in jobs:
        upload_duration = (B[i] / Cu) * gp.quicksum(Y[i, r] for r in R_i[i])
        m.addConstr(t_e_up[i] == t_s_up[i] + upload_duration, f"TimeRel_Up_{i}")

    # Execution duration
    for i in jobs:
        mec_duration = E[i] * gp.quicksum(Z[i, x] for x in servers)
        local_duration = gamma[i] * E[i] * Y_local[i]
        m.addConstr(t_e_ex[i] == t_s_ex[i] + mec_duration + local_duration, f"TimeRel_Ex_{i}")

    # -- Assignment Constraints --
    
    # Each job is either offloaded (via one RSU) or executed locally
    m.addConstrs((gp.quicksum(Y[i, r] for r in R_i[i]) + Y_local[i] == 1 
                  for i in jobs), "Assign_RSU_or_Local")
    
    # Each job either executes on one MEC server or executes locally
    m.addConstrs((Z.sum(i, '*') + Y_local[i] == 1 
                  for i in jobs), "Assign_Server_or_Local")

    # -- Sequential Dependency --
    
    # Execution starts after upload, but only if an upload happened
    for i in jobs:
        is_offloaded = gp.quicksum(Y[i, r] for r in R_i[i])
        # If local (is_offloaded=0), constraint becomes t_s_ex >= t_e_up - M
        m.addConstr(t_s_ex[i] >= t_e_up[i] - M * (1 - is_offloaded), f"Sequential_{i}")

    # -- Non-Overlap Constraints (Activated Big-M) --
    
    # RSU non-overlap
    for r in rsus:
        for i in jobs:
            for j in jobs:
                if i < j:
                    if (i, j, r) in z_up.keys():
                        z = z_up[i, j, r]
                        d_ij = delta_up[i, j, r]
                        d_ji = delta_up[j, i, r]
                        Yi = Y[i, r]
                        Yj = Y[j, r]
                        
                        # Activation
                        m.addConstr(z <= Yi, f"RSU_Act_{i}_{j}_{r}_a")
                        m.addConstr(z <= Yj, f"RSU_Act_{i}_{j}_{r}_b")
                        m.addConstr(z >= Yi + Yj - 1, f"RSU_Act_{i}_{j}_{r}_c")
                        
                        # Ordering
                        m.addConstr(t_e_up[i] + epsilon <= t_s_up[j] + M * (1 - d_ij) + M * (1 - z),
                                    f"RSU_NoOverlap_{i}_{j}_{r}_ibefj")
                        m.addConstr(t_e_up[j] + epsilon <= t_s_up[i] + M * (1 - d_ji) + M * (1 - z),
                                    f"RSU_NoOverlap_{i}_{j}_{r}_jbefi")
                        
                        # Exactly-one ordering if active
                        m.addConstr(d_ij + d_ji == z, f"RSU_Order_{i}_{j}_{r}")

    # Server non-overlap
    for x in servers:
        for i in jobs:
            for j in jobs:
                if i < j:
                    z = z_ex[i, j, x]
                    d_ij = delta_ex[i, j, x]
                    d_ji = delta_ex[j, i, x]
                    Zi = Z[i, x]
                    Zj = Z[j, x]
                    
                    # Activation
                    m.addConstr(z <= Zi, f"Server_Act_{i}_{j}_{x}_a")
                    m.addConstr(z <= Zj, f"Server_Act_{i}_{j}_{x}_b")
                    m.addConstr(z >= Zi + Zj - 1, f"Server_Act_{i}_{j}_{x}_c")
                    
                    # Ordering
                    m.addConstr(t_e_ex[i] + epsilon <= t_s_ex[j] + M * (1 - d_ij) + M * (1 - z),
                                f"Server_NoOverlap_{i}_{j}_{x}_ibefj")
                    m.addConstr(t_e_ex[j] + epsilon <= t_s_ex[i] + M * (1 - d_ji) + M * (1 - z),
                                f"Server_NoOverlap_{i}_{j}_{x}_jbefi")
                    
                    # Exactly-one ordering if active
                    m.addConstr(d_ij + d_ji == z, f"Server_Order_{i}_{j}_{x}")

    # -- Deadline Constraints --
    m.addConstrs((t_e_ex[i] <= D[i] for i in J_RT), "Deadline")

    # --- 5. Objective Function ---
    
    # Term 1: Weighted total completion time (latency)
    term1 = alpha * gp.quicksum(I[i] * t_e_ex[i] for i in jobs)
    
    # Term 2: Total computation cost (only for MEC execution)
    term2_cost = beta * gp.quicksum(B[i] * c_s * Z[i, x] 
                                    for i in jobs for x in servers)

    m.setObjective(term1 + term2_cost, GRB.MINIMIZE)

    # --- 6. Optimize ---
    m.optimize()

    # --- 7. Process Results ---
    results = {}
    if m.Status == GRB.OPTIMAL:
        print(f"\nOptimal solution found! Objective value: {m.ObjVal:.4f}")
        results['ObjVal'] = m.ObjVal
        results['Assignments'] = []
        results['Schedule'] = []

        for i in jobs:
            is_local = Y_local[i].X > 0.5
            
            rsu_assigned = -1
            server_assigned = -1
            
            if not is_local:
                for r in R_i[i]:
                    if (i, r) in Y and Y[i, r].X > 0.5:
                        rsu_assigned = r
                        break
                for x in servers:
                    if Z[i, x].X > 0.5:
                        server_assigned = x
                        break
            
            job_res = {
                'Job': i,
                'Local': is_local,
                'RSU': rsu_assigned,
                'Server': server_assigned
            }
            results['Assignments'].append(job_res)
            
            sched_res = {
                'Job': i,
                'Upload_Start': t_s_up[i].X,
                'Upload_End': t_e_up[i].X,
                'Exec_Start': t_s_ex[i].X,
                'Exec_End': t_e_ex[i].X,
            }
            results['Schedule'].append(sched_res)
            
            # Print friendly output
            print(f"\nJob {i}:")
            if is_local:
                print(f"  Assigned: Local Execution")
            else:
                print(f"  Assigned RSU: {rsu_assigned}, Assigned Server: {server_assigned}")
            
            upload_dur = t_e_up[i].X - t_s_up[i].X
            exec_dur = t_e_ex[i].X - t_s_ex[i].X
            
            print(f"  Upload:   [{t_s_up[i].X:.2f}s - {t_e_up[i].X:.2f}s] (Duration: {upload_dur:.2f}s)")
            print(f"  Execute:  [{t_s_ex[i].X:.2f}s - {t_e_ex[i].X:.2f}s] (Duration: {exec_dur:.2f}s)")
            
            if i in J_RT:
                met_deadline = t_e_ex[i].X <= D[i] + 1e-6 # Add tolerance for float comparison
                print(f"  Deadline: {D[i]:.2f}s (Met: {met_deadline})")

    elif m.Status == GRB.INFEASIBLE:
        print("\nModel is infeasible. No solution exists.")
        print("Computing IIS (Irreducible Inconsistent Subsystem) to find conflicting constraints...")
        m.computeIIS()
        m.write("model_iis.ilp")
        print("IIS written to model_iis.ilp")
        results = None
    else:
        print(f"\nOptimization ended with status: {m.Status}")
        results = None

    return results

# --- Main execution block to run a heavier example ---
# --- Main execution block to run an example ---
if __name__ == "__main__":
    
    # --- Define Sets ---
    J = list(range(1, 11))   # 10 jobs
    R = [1, 2, 3]            # 3 RSUs
    S = [1, 2]               # 2 MEC servers
    J_RT = [1, 3, 5, 7, 9]    # some RT jobs

    print("--- Starting RSU/Server/Local Allocation Optimization (10-JOB INSTANCE) ---")

    # --- Parameters ---
    params = {}
    
    # Reachable RSUs for each job i (pattern ensures variety)
    params['R_i'] = {
        i: [((i % 3) + 1), (((i + 1) % 3) + 1)]
        for i in J
    }

    # Data size (Mbits) – moderate but varied
    params['B'] = {
        i: 200 + (i * 30)   # grows: 230, 260, ..., 500
        for i in J
    }

    # MEC Execution time (seconds)
    params['E'] = {
        i: 2 + (i % 4)       # cycles through 2,3,4,5
        for i in J
    }

    # Deadlines (only RT jobs)
    params['D'] = {
        i: 80 + (i * 5)      # big enough to avoid infeasibility
        for i in J_RT
    }

    # Importance factor (random-like pattern)
    params['I'] = {
        i: 1 + (i % 7)
        for i in J
    }

    # RSU upload capacity
    params['Cu'] = 50.0

    # MEC computation cost
    params['c_s'] = 0.05

    # Local slowdown factor
    params['gamma'] = {i: 10.0 for i in J}

    # Objective weights
    params['alpha'] = 2.0
    params['beta'] = 1.0

    # Big-M and epsilon
    params['M'] = 2000
    params['epsilon'] = 0.001

    # --- Call Solver ---
    solution = solve_rsu_allocation_with_local(J, R, S, J_RT, params)

    if solution:
        print("\n--- Optimization Complete ---")
    else:
        print("\n--- Optimization Failed ---")


--- Starting RSU/Server/Local Allocation Optimization (10-JOB INSTANCE) ---
Set parameter Username
Set parameter LicenseID to value 2718223
Academic license - for non-commercial use only - expires 2026-10-06

Optimal solution found! Objective value: 1315.3700

Job 1:
  Assigned RSU: 3, Assigned Server: 2
  Upload:   [11.60s - 16.20s] (Duration: 4.60s)
  Execute:  [16.20s - 19.20s] (Duration: 3.00s)
  Deadline: 85.00s (Met: True)

Job 2:
  Assigned RSU: 3, Assigned Server: 2
  Upload:   [6.40s - 11.60s] (Duration: 5.20s)
  Execute:  [11.60s - 15.60s] (Duration: 4.00s)

Job 3:
  Assigned RSU: 1, Assigned Server: 1
  Upload:   [7.00s - 12.80s] (Duration: 5.80s)
  Execute:  [12.80s - 17.80s] (Duration: 5.00s)
  Deadline: 95.00s (Met: True)

Job 4:
  Assigned RSU: 3, Assigned Server: 1
  Upload:   [0.00s - 6.40s] (Duration: 6.40s)
  Execute:  [6.40s - 8.40s] (Duration: 2.00s)

Job 5:
  Assigned RSU: 1, Assigned Server: 2
  Upload:   [0.00s - 7.00s] (Duration: 7.00s)
  Execute:  [7.00s - 10.